## Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

## Load data and plot

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
df = pd.read_csv(url)

y = df["Passengers"].astype("float32").to_numpy()

plt.figure()
plt.plot(y)
plt.title("Passenger-data")
plt.xlabel("Month")
plt.ylabel("Passengers")
plt.show()

## Normalize

In [ ]:
y_min, y_max = y.min(), y.max()
y = (y - y_min) / (y_max - y_min)   # now y is 0..1

## Make prediction windows

In [ ]:
W = 20  # TODO: choose window size
X, Y = [], []
for i in range(len(y) - W):
    X.append(y[i:i+W])
    Y.append(y[i+W])

X = np.array(X, dtype="float32")            # (N, W)
Y = np.array(Y, dtype="float32")            # (N,)
X = X[..., None]                             # (N, W, 1) for Keras

## (Optional) Plot sample window

In [ ]:
plt.figure()
plt.plot(X[0].squeeze())
plt.title("Example window (model input)")
plt.show()

## Train/val/test split

In [ ]:
n = len(X)
n_train = int(n * 0.7)
n_val = int(n * 0.15)

X_train, Y_train = X[:n_train], Y[:n_train]
X_val,   Y_val   = X[n_train:n_train+n_val], Y[n_train:n_train+n_val]
X_test,  Y_test  = X[n_train+n_val:], Y[n_train+n_val:]

## Build model

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(W, 1)), # Input layer with same size as data
    
    layers.Dense(1) # Predicting one value
])

## Compile with optimizer

In [ ]:
model.compile(
    optimizer="adam", 
    loss="mse", 
    metrics=["mae"]
    )

In [ ]:
model.summary()

## Train (yay)

In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=10,                     
    batch_size=32,
    verbose=1
)

## Predict and "de-normalize"

In [ ]:
test_loss, test_mae = model.evaluate(X_test, Y_test, verbose=0)
print("Test MAE:", test_mae)

pred = model.predict(X_test, verbose=0).squeeze()

pred = pred * (y_max - y_min) + y_min
Y_test = Y_test * (y_max - y_min) + y_min

## Plot

In [ ]:
plt.plot(Y_test[:200], label="true")
plt.plot(pred[:200], label="pred")
plt.legend()
plt.title("Next-step prediction (first 200 test points)")
plt.show()